In [1]:
import os
import random
import csv
from glob import glob
from collections import defaultdict

import numpy as np
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import cv2


# -------------------- Load Character Images --------------------

def load_char_images(dataset_dir, exts=("png","jpg","jpeg")):
    char_map = defaultdict(list)
    for entry in os.listdir(dataset_dir):
        folder = os.path.join(dataset_dir, entry)
        if not os.path.isdir(folder):
            continue
        char = entry
        for ext in exts:
            for path in glob(os.path.join(folder, f"*.{ext}")):
                try:
                    img = Image.open(path).convert("RGBA")
                    char_map[char].append(img)
                except:
                    pass
    if not char_map:
        raise ValueError("No character folders found!")
    return dict(char_map)



# -------------------- Background Generator --------------------

def random_paper_background(w, h):
    base = np.ones((h, w, 3), dtype=np.uint8) * random.randint(230, 245)
    speckles = (np.random.randn(h, w) * random.uniform(1, 4)).astype(np.int16)

    for c in range(3):
        base[..., c] = np.clip(base[..., c] + speckles, 0, 255)

    bg = Image.fromarray(base).filter(
        ImageFilter.GaussianBlur(radius=random.uniform(0.2, 0.6))
    )
    return bg



# -------------------- Mild Global Augmentations --------------------

def apply_global_augmentations(img,
                               max_rotation=1.0,
                               perspective_jitter=0.02,
                               brightness_jitter=0.10,
                               contrast_jitter=0.10,
                               blur_prob=0.10,
                               noise_level=3):

    w, h = img.size

    # tiny rotation
    angle = random.uniform(-max_rotation, max_rotation)
    img = img.rotate(angle, resample=Image.NEAREST, expand=False, fillcolor=(255,255,255))

    # tiny perspective
    if random.random() < 0.25:
        src = np.float32([[0,0],[w,0],[w,h],[0,h]])
        mx = perspective_jitter * min(w,h)
        jitter = lambda m: random.uniform(-m, m)
        dst = np.float32([
            [jitter(mx), jitter(mx)],
            [w + jitter(mx), jitter(mx)],
            [w + jitter(mx), h + jitter(mx)],
            [jitter(mx), h + jitter(mx)]
        ])
        M = cv2.getPerspectiveTransform(src, dst)
        arr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        arr = cv2.warpPerspective(arr, M, (w,h), borderMode=cv2.BORDER_REPLICATE)
        img = Image.fromarray(cv2.cvtColor(arr, cv2.COLOR_BGR2RGB))

    # brightness + contrast
    img = ImageEnhance.Brightness(img).enhance(
        random.uniform(1-brightness_jitter, 1+brightness_jitter)
    )
    img = ImageEnhance.Contrast(img).enhance(
        random.uniform(1-contrast_jitter, 1+contrast_jitter)
    )

    # small blur
    if random.random() < blur_prob:
        img = img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 0.5)))

    # small noise
    if random.random() < 0.30:
        arr = np.array(img).astype(np.int16)
        noise = (np.random.randn(*arr.shape) * noise_level).astype(np.int16)
        arr = np.clip(arr + noise, 0, 255).astype(np.uint8)
        img = Image.fromarray(arr)

    return img



# -------------------- Word Stitcher (FIXED) --------------------

def stitch_char_images_for_word(word,
                                char_map,
                                target_height=120,
                                per_char_scale_jitter=(0.98,1.03),
                                per_char_rot_jitter=1.0,
                                spacing_range=(6,14),
                                vertical_jitter=3):

    char_imgs = []
    for ch in word:
        if ch not in char_map:
            raise KeyError(f"Character '{ch}' not found!")

        img = random.choice(char_map[ch]).copy()

        # keep dot crisp: scale with NEAREST
        s = random.uniform(*per_char_scale_jitter)
        h = int(target_height * s)
        w = int(img.width * (h / img.height))
        img = img.resize((w, h), resample=Image.NEAREST)

        # tiny rotation with NEAREST to prevent blur
        angle = random.uniform(-per_char_rot_jitter, per_char_rot_jitter)
        img = img.rotate(angle, expand=True, resample=Image.NEAREST,
                         fillcolor=(0,0,0,0))

        char_imgs.append(img)

    # compute total width
    spacing = [random.randint(*spacing_range) for _ in range(len(char_imgs)-1)]
    total_w = sum(im.width for im in char_imgs) + sum(spacing)
    max_h = max(im.height for im in char_imgs)

    canvas = Image.new("RGBA", (total_w, max_h + 2*vertical_jitter), (0,0,0,0))

    x = 0
    for i, im in enumerate(char_imgs):
        y = (canvas.height - im.height)//2 + random.randint(-vertical_jitter, vertical_jitter)
        canvas.alpha_composite(im, (x, y))
        x += im.width
        if i < len(spacing):
            x += spacing[i]

    return canvas



# -------------------- Main Generator --------------------

DEFAULT_WORD_LIST = [
    "BOX","FOX","MIX","QUICK","QUEEN","QUIET","ZEBRA","QUIZ","XRAY",
    "THE","AND","YOU","FOR","ARE","BUT","NOT","CAN","ONE","ALL","SHE","HE",
    "HIS","HER","WAS","THEY","THEM","THIS","THAT","WITH","HAVE","FROM","LIKE",
    "GET","JUST","MAKE","TAKE","GOOD","GREAT","WHERE","WHICH","WHEN","WHAT",
    "HOW","WHY","OVER","INTO","EVERY","AFTER","AGAIN","BECAUSE","BEFORE",
    "AROUND","PEOPLE","RIGHT","STILL","GOING","WANT","NEED","JAZZ","QUIT",
    "CAT","DOG","BALL","BOOK","PEN","BAG","PHONE","TABLE","CHAIR","DOOR",
    "FOOD","WATER","HOUSE","SCHOOL","CAR","BUS","TREE","FLOWER","STREET",
    "LIGHT","SOUND","FRUIT","BREAD","MONEY","ANIMAL","ZOO","ZERO","ZIP",
    "FAMILY","FATHER","MOTHER","CHILDREN","FRIEND","NUMBER","SIMPLE","LETTER",
    "SYSTEM","COUNTRY","IMPORTANT","DIFFERENT","ANYTHING","EVERYTHING",
    "SOMETHING","TOGETHER","WITHOUT","BETWEEN","BELIEVE","FOLLOW","UNDER",
    "UNTIL","ABOUT","WORLD"
]


def synthesize_given_words(dataset_dir,
                           output_dir,
                           word_list=DEFAULT_WORD_LIST,
                           per_word_count=200,
                           target_height=120,
                           seed=1337):

    random.seed(seed)
    np.random.seed(seed)

    os.makedirs(output_dir, exist_ok=True)

    char_map = load_char_images(dataset_dir)

    for w in word_list:

        folder = os.path.join(output_dir, w)
        os.makedirs(folder, exist_ok=True)
        csv_path = os.path.join(folder, "labels.csv")

        print(f"Generating {per_word_count} images for '{w}'")

        with open(csv_path, "w", newline="", encoding="utf-8") as csv_f:
            writer = csv.writer(csv_f)
            writer.writerow(["filename", "word"])

            for i in range(per_word_count):

                rgba = stitch_char_images_for_word(w, char_map, target_height)

                bg = random_paper_background(max(200, rgba.width+40), rgba.height+40)
                bg = bg.convert("RGBA")

                x = random.randint(10, 30)
                y = (bg.height - rgba.height)//2 + random.randint(-4,4)

                bg.alpha_composite(rgba, (x, y))
                composed = bg.convert("RGB")

                final = apply_global_augmentations(composed)

                fname = f"{w}_{i:05d}.png"
                final.save(os.path.join(folder, fname))
                writer.writerow([fname, w])

                if (i+1) % 50 == 0:
                    print(" ", i+1, "/", per_word_count)

    print("DONE.")



# -------------------- Run --------------------

if __name__ == "__main__":
    dataset_dir = r"C:\new braille\Braille word"
    output_dir = r"Clean_braille_words_fixed"
    per_word_count = 200

    synthesize_given_words(dataset_dir, output_dir,
                           word_list=DEFAULT_WORD_LIST,
                           per_word_count=per_word_count,
                           target_height=120,
                           seed=42)


Generating 200 images for 'BOX'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'FOX'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'MIX'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'QUICK'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'QUEEN'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'QUIET'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'ZEBRA'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'QUIZ'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'XRAY'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'THE'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'AND'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'YOU'
  50 / 200
  100 / 200
  150 / 200
  200 / 200
Generating 200 images for 'FOR'
  50 / 200